In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Dataset paths
data_dir = './dataset/cell_images'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')


In [ ]:
os.listdir(train_dir)

In [ ]:
len(os.listdir(train_dir+'/uninfected/'))

In [ ]:
len(os.listdir(train_dir+'/parasitized/'))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [ ]:
# Checking the image dimensions
dim1 = []
dim2 = []
for image_filename in os.listdir(test_dir+'/uninfected'):
    img = imread(test_dir+'/uninfected'+'/'+image_filename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
sns.jointplot(x=dim1,y=dim2)

In [ ]:
# Parameters
image_size = (130, 130)
batch_size = 16
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Data transforms
train_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, shear=10, scale=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])


In [ ]:
# Data loaders
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [ ]:
# Write your code here

class CNNClassifier(nn.Module):
    def __init__(self, input_channels=3, input_height=130, input_width=130):
        super(CNNClassifier, self).__init__()
        # Write your code here





    def forward(self, x):
        # Write your code here





In [ ]:
model = CNNClassifier(input_channels=3, input_height=130, input_width=130).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())


In [ ]:
# Function to Train the Model
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    train_losses, val_losses = [], []
    for epoch in range(epochs):
        model.train()
        # write Your code here







        model.eval()
        # write Your code here








        print(f"Epoch [{epoch+1}/{epochs}] → Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

    # Plot loss
    print('Name:             ')
    print('Register Number:         ')
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()
    return train_losses, val_losses


In [ ]:
# Run the experiment
# Train the model
# Write Your Code Here


torch.save(model.state_dict(), 'cell_model.pth')


In [ ]:
import seaborn as sns

In [ ]:
def evaluate_model(model, test_loader, test_data):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device).float().unsqueeze(1)  # Ensure float for binary targets
            output = model(X)
            predicted = (output > 0.5).int()  # Binary prediction
            total += y.size(0)
            correct += (predicted == y.int()).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    class_names = test_data.classes  # ['parasitized', 'uninfected']

    print('Name:             ')
    print('Register Number:         ')
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    # Classification Report
    print('Name:             ')
    print('Register Number:         ')
    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))



In [ ]:
evaluate_model(model, test_loader, test_data)

In [ ]:
import matplotlib.pyplot as plt

def predict_image(model, image_index, dataset, image_size=(130, 130)):
    model.eval()
    image, label = dataset[image_index]  # image is already transformed
    with torch.no_grad():
        output = model(image.unsqueeze(0).to(device))  # Add batch dimension
        predicted = (output > 0.5).int()

    class_names = dataset.classes  # ['parasitized', 'uninfected']

    # Convert tensor image to numpy for plotting
    img_np = image.permute(1, 2, 0).cpu().numpy()  # (C, H, W) -> (H, W, C)

    # Display the image
    print('Name:             ')
    print('Register Number:         ')
    plt.imshow(img_np)
    plt.title(f'Actual: {class_names[label]}\nPredicted: {class_names[predicted.item()]}')
    plt.axis("off")
    plt.show()



In [ ]:
predict_image(model, image_index=5, dataset=test_data)

In [ ]:
predict_image(model, image_index=1500, dataset=test_data)

In [ ]:
predict_image(model, image_index=1800, dataset=test_data)